<div style="line-height:0.5">
<h1 style="color:darkkhaki"> SQLite basics 3 </h1>
<h4> Common queries and operations. Focus on joins </h4>
<span style="display: inline-block;">
    <h3 style="color: lightblue; display: inline;">Keywords:</h3> placeholders + rowid + save + limit 1
</span>
</div>
<br>
<div style="line-height:1.2">
<span style="display: inline-block;">
    <h3 style="color: red; display: inline;">Notes:</h3> 
    The following examples are not reproducible. In fact, due to its large size, the database was not uploaded. <br>
    &emsp;&emsp;&emsp;&emsp;&emsp;Look at sqlite_2_trials.ipynb for further info. <br>
    &emsp;&emsp;&emsp;&emsp;&emsp;For same reasons the generated file "resp_query6.csv" was not uploaded.
</span>
</div>

In [2]:
import csv
import random
import linecache 
import sqlite3 as lite
import pandas as pd

<h2 style="color:darkkhaki"> #1  Fetch data from db  </h2>

In [3]:
conn = lite.connect('./databases_used_in_examples/oimi_robot_database.db')
curs = conn.cursor()
p1 = 'ask_quest_20'
p2 = 'kind_game_9C'
p3 = 'choose three_games'
p3 = 'choose four_games'
params = (p1,)

In [3]:
curs.execute("SELECT Sessions.session_id FROM Sessions JOIN Sessions_Impositions USING (session_id) \
    WHERE (status ='default' OR status='to_do') \
    AND Sessions_Impositions.imposition_name = ?\
    ORDER BY random()", params)

result_sess = curs.fetchall()
res = [item for sublist in result_sess for item in sublist]
res = list(set(res))
print(res)

[20004]


In [4]:
params = (p2,)
curs.execute("SELECT Sessions.session_id FROM Sessions JOIN Sessions_Impositions USING (session_id) \
    WHERE (status ='default' OR status='to_do') \
    AND Sessions_Impositions.imposition_name = ?\
    ORDER BY random()",params)
result_sess1 = curs.fetchall()
res1 = [item for sublist in result_sess1 for item in sublist]
res1 = list(set(res1))
print(res1)

[20011]


In [5]:
session_and = list(set(res) & (set(res1)))
session_or = list(set(res) | (set(res1)))

print(session_and)
print(session_or)

[]
[20011, 20004]


In [6]:
""" Check common values """
a = [84, 534 ,4, 24, 783]
b = [4, 33 ,6, 534, 24, 98]
c = [4, 77 ,56, 24]

oo = list(set(a) & set(b) & set(c))
print(oo)


[24, 4]


<h2 style="color:darkkhaki"> #2 Visualize output with Pandas  </h2>

In [7]:
conn = lite.connect("./databases_used_in_examples/oimi_robot_database.db")
curs = conn.cursor()

kid_id = 10000
#query_take = "SELECT name,surname,age,current_level FROM Kids WHERE kid_id={}".format(kid_id)
query_take = "SELECT name,surname,age,current_level FROM Kids".format(kid_id)
df = pd.read_sql_query(query_take, conn)

print(df.head())

       name  surname  age current_level
0     Andre   Ampere    6    elementary
1   Camillo    Golgi    7      beginner
2  Giuseppe   Ponzio    9  intermediate
3  Giovanni  Celoria    8    elementary
4   Edoardo  Bassini   10      advanced


<h2 style="color:darkkhaki"> #3 Utilize placeholders </h2>

In [13]:
# single selection
selections = [10]
place_holders = ",".join("?"*len(selections))
ququ = "select kind, type, audio, value from Questions where question_id in (?)"
curs.execute(ququ, selections)
return_que = curs.fetchall() 

print("1 return_que return_que are {}".format(return_que))

1 return_que return_que are [('1F', '1FSUS', 'audio_10', 1)]


In [18]:
# multi selection
selections = [10, 11, 12, 13, 14, 15]
place_holders = ",".join("?" * len(selections))
ququ = f"select kind, type, audio, value from Questions where question_id in ({place_holders})"
curs.execute(ququ, selections)
return_que = curs.fetchall()

print("return_que:")
return_que

return_que:


[('1F', '1FSUS', 'audio_10', 1),
 ('1F', '1FSUS', 'audio_11', 1),
 ('1F', '1FSUS', 'audio_12', 1),
 ('1F', '1FSUS', 'audio_13', 1),
 ('1F', '1FSUS', 'audio_14', 1),
 ('1F', '1FSUD', 'audio_15', 1)]

In [21]:
ki = return_que[0][0]
ty = return_que[0][1]
au = return_que[0][2]
val = return_que[0][3]
res = 0

print(ki)
print(ty)
print(au)
print(val)
print(res)
print(type(return_que))

1F
1FSUS
audio_10
1
0
<class 'list'>


In [29]:
""" Search for data in query results.
1) Convert to tuple before executing;
2) Exit the inner loop once found;
N.B.
Data returned from the database has one or more integer values in the tuple.
It may happens => "TypeError: argument of type 'int' is not iterable" error.
To handle the error, check with "isinstance" whether the element in the tuple is an integer before attempting to iterate over it. 
If so =>  convert it to a string 
"""
curs.execute(ququ, tuple(selections))
return_que = curs.fetchall()
print("1 return_que return_que are {}".format(return_que))

to_search = '1FSUS'
res = 0 

for i in return_que:
    print(i)

    for j in range(len(i)):
        if isinstance(i[j], int):
            i[j] = str(i[j])

        if to_search in i[j]:
            print("Found in tuple:", i)
            res = 1
            break

if res == 1:
    print("Found at least once.")
else:
    print("Not found.")

1 return_que return_que are [('1F', '1FSUS', 'audio_10', 1)]
('1F', '1FSUS', 'audio_10', 1)
Found in tuple: ('1F', '1FSUS', 'audio_10', 1)
Found at least once.


In [8]:
def get_line_selection():
    read_line = linecache.getline('./queries_used_in_examples/text_to_get.py', 1)
    return read_line

query_string_3 = '''select & from ? where ! in ({})'''

prima_col = 'name'
sec_col = 'Kids'
ter_col = 'current_level'

fine = "{}, {}, {}".format(prima_col, sec_col, ter_col)

leggiline = get_line_selection()

res = leggiline.replace("&", prima_col)
res1 = res.replace("?", sec_col)
res2 = res1.replace("!", ter_col)

print(fine)
print(leggiline)
print(res)
print(res1)
print(res2)

name, Kids, current_level
select & from ? where ! in ('elementary')

select name from ? where ! in ('elementary')

select name from Kids where ! in ('elementary')

select name from Kids where current_level in ('elementary')



In [9]:
curs.execute(res2)
rows = curs.fetchall()

for row in rows:
    print(row[0])

Andre
Giovanni


In [12]:
sql_joint = "SELECT COUNT(rowid) FROM Scenarios_Questions" 

curs.execute(sql_joint)
#res = curs.fetchall()
#res = curs.fetchone()
res = curs.fetchone()[0]
#risp = res[0]
print(res)
##print(risp)

curs.execute('''SELECT scenario_id FROM Scenarios WHERE scenario_group IN ('Scenario_3045','Scenario_5175','Scenario_4721', 'Scenario_2003',
'Scenario_379','Scenario_2739','Scenario_835') ORDER BY RANDOM() LIMIT 1''')

scenarioo = curs.fetchone()[0]
print(scenarioo)


3054743
6008601160066007


<h2 style="color:darkkhaki"> #4 Count </h2>

In [13]:
# Get Session table's single last update
curs.execute("SELECT session_id FROM Sessions ORDER BY session_id DESC LIMIT 1")
sess_last = curs.fetchone()[0]
print("sess last {}".format(sess_last))

curs.execute("SELECT COUNT(session_id) FROM Kids_Sessions WHERE session_id = ?",(sess_last,))
num_session_of_current_kid =curs.fetchone()[0]
print(num_session_of_current_kid)

curs.execute("SELECT match_id FROM Matches ORDER BY match_id DESC LIMIT 1")
match_last = curs.fetchone()[0]
print("match last for generating audio_intro e audio_exit session is == {}".format(match_last))
curs.execute('''SELECT COUNT(p.patch_id) FROM Patches p 
			JOIN (Scenarios s JOIN Matches m ON s.scenario_id=m.scenario)
			ON p.patch_id=s.patch1 OR p.patch_id=s.patch2 OR p.patch_id=s.patch3 OR p.patch_id=s.patch4
			WHERE genre="Animal" AND match_id = ?''',(match_last,))
pa_gen_ani = curs.fetchone()[0]
print(pa_gen_ani)
curs.execute('''SELECT COUNT(p.patch_id) FROM Patches p 
			JOIN (Scenarios s JOIN Matches m ON s.scenario_id=m.scenario)
			ON p.patch_id=s.patch1 OR p.patch_id=s.patch2 OR p.patch_id=s.patch3 OR p.patch_id=s.patch4
			WHERE genre="Solid Color" AND match_id = ?''',(match_last,))
pa_gen_color = curs.fetchone()[0]
print(pa_gen_color)
curs.execute('''SELECT COUNT(p.patch_id) FROM Patches p 
			JOIN (Scenarios s JOIN Matches m ON s.scenario_id=m.scenario)
			ON p.patch_id=s.patch1 OR p.patch_id=s.patch2 OR p.patch_id=s.patch3 OR p.patch_id=s.patch4
			WHERE genre="People" AND match_id = ?''',(match_last,))
pa_gen_peo = curs.fetchone()[0]
print(pa_gen_peo)
curs.execute('''SELECT COUNT(p.patch_id) FROM Patches p 
			JOIN (Scenarios s JOIN Matches m ON s.scenario_id=m.scenario)
			ON p.patch_id=s.patch1 OR p.patch_id=s.patch2 OR p.patch_id=s.patch3 OR p.patch_id=s.patch4
			WHERE genre="Dress" AND match_id = ?''',(match_last,))
pa_gen_dress = curs.fetchone()[0]
print(pa_gen_dress)
curs.execute('''SELECT COUNT(p.patch_id) FROM Patches p 
			JOIN (Scenarios s JOIN Matches m ON s.scenario_id=m.scenario)
			ON p.patch_id=s.patch1 OR p.patch_id=s.patch2 OR p.patch_id=s.patch3 OR p.patch_id=s.patch4
			WHERE genre="Thing" AND match_id = ?''',(match_last,))
pa_gen_thi = curs.fetchone()[0]
print(pa_gen_thi)
curs.execute('''SELECT COUNT(p.patch_id) FROM Patches p 
			JOIN (Scenarios s JOIN Matches m ON s.scenario_id=m.scenario)
			ON p.patch_id=s.patch1 OR p.patch_id=s.patch2 OR p.patch_id=s.patch3 OR p.patch_id=s.patch4
			WHERE genre="Food" AND match_id = ?''',(match_last,))
pa_gen_food = curs.fetchone()[0]
print(pa_gen_food)

sess last 20022
1
match last for generating audio_intro e audio_exit session is == 30053
3
0
0
0
1
0


In [17]:
print(type(sess_last))
#curs.execute("SELECT match_id FROM Sessions_Matches WHERE session_id =2001")
curs.execute("SELECT match_id FROM Sessions_Matches WHERE session_id =?", (sess_last,))
mama = curs.fetchall()
matches_last = [item for sublist in mama for item in sublist]
print()
print("Here is :  {}".format(matches_last))
lista_pa = []
for i in range(len(matches_last)):
    print("We are in round num {}, audio_intro e audio_exit session are == {}".format(i, matches_last))
    curs.execute('''SELECT COUNT(p.patch_id) FROM Patches p 
        JOIN (Scenarios s JOIN Matches m ON s.scenario_id=m.scenario)
        ON p.patch_id=s.patch1 OR p.patch_id=s.patch2 OR p.patch_id=s.patch3 OR p.patch_id=s.patch4
        WHERE genre="Animal" AND match_id = ?''',(matches_last[i],))
    pa_gen_ani = curs.fetchone()[0]
    print("pa_gen_ani is {}", format(pa_gen_ani))
    lista_pa.append(pa_gen_ani)
print("lista",lista_pa)

<class 'int'>

Here is :  [30051, 30052, 30053]
We are in round num 0, audio_intro e audio_exit session are == [30051, 30052, 30053]
pa_gen_ani is {} 0
We are in round num 1, audio_intro e audio_exit session are == [30051, 30052, 30053]
pa_gen_ani is {} 3
We are in round num 2, audio_intro e audio_exit session are == [30051, 30052, 30053]
pa_gen_ani is {} 3
lista [0, 3, 3]


<h2 style="color:darkkhaki"> #5 Triple join </h2>

In [6]:
group = 'Scenario_4721'
this_kind = '1F'
this_type = '1FCACO'
limit = 1

params = (group, this_kind, this_type, limit)

sql_q = "SELECT question_id, audio, kind, type, description, value \
FROM Scenarios_Questions \
JOIN (Questions q \
JOIN Games_Questions using (question_id)) ON audio_id = audio \
WHERE scenario_group = 'Scenario_4721' AND kind = '1F' AND type = '1FCACO' \
GROUP BY RANDOM() \
LIMIT 3;"

sql_q_2 = "SELECT question_id, audio, kind, type, description, value \
FROM Scenarios_Questions \
JOIN Questions q ON audio_id = audio \
WHERE scenario_group = 'Scenario_4721' AND kind = '1F' AND type = '1FCACO' \
GROUP BY RANDOM() \
LIMIT 1;"

curs.execute(sql_q)
#curs.execute(sql_q, (params,))
res = curs.fetchall()
res

[(1763, 'audio_1763', '1F', '1FCACO', 'Tocca un animale rosso', 2),
 (1764, 'audio_1764', '1F', '1FCACO', 'Tocca un animale verde', 2),
 (1764, 'audio_1764', '1F', '1FCACO', 'Tocca un animale verde', 2)]

<h2 style="color:darkkhaki"> #6 Copy query results in a file with csv format </h2>

In [7]:
queque = "SELECT scenario_group, type FROM Scenarios_Questions sq JOIN Questions q ON sq.audio_id=q.audio ORDER BY scenario_group"
db_df = pd.read_sql_query(queque, conn)
db_df.to_csv('./datasets_used_in_examples/resp_query6.csv', index=False)